# Constructing the full set of graphs

Import libraries

In [ ]:
import networkx as nx
import numpy as np
import pandas as pd
import torch

In [ ]:
def torch_pearson_correlation_matrix(X):

  mean = torch.mean(X, dim=0)
  covariance = torch.matmul((X - mean).t(), (X - mean))
  variance = torch.diagonal(covariance)
  correlation = (variance**(-1/2)) * covariance * ((variance**(-1/2)).t())
  return correlation

def construct_graph(roi_time_series, threshold=0.7, return_functional_connectivity=True, adjacency_matrix_type='torch', adjacency_representation = 'edge_list'):

  roi_time_series = torch.from_numpy(roi_time_series)
  functional_connectivity = torch_pearson_correlation_matrix(roi_time_series)
  adjacency = (functional_connectivity > threshold).int() - torch.eye(len(functional_connectivity))

  if adjacency_representation == 'edge_list':
    adjacency = (adjacency.nonzero()).t()

  if adjacency_matrix_type == 'numpy':
    adjacency = adjacency.numpy()

  if return_functional_connectivity==True:
    return adjacency, functional_connectivity
  else:
    return adjacency

Obtaining data from the ABIDE I [Preprocessed Connectomes Project](http://preprocessed-connectomes-project.org/abide/download.html). We're going to use the AAL atlas, which uses 116 brain regions of interest.

In [ ]:
import os
import subprocess

def download_sample(sample_name):
  destination_folder = '/content/data/fmri'
  url = 'https://s3.amazonaws.com/fcp-indi/data/Projects/ABIDE_Initiative/Outputs/cpac/filt_global/rois_aal/{}_rois_aal.1D'.format(sample_name)
  subprocess.run([
    "wget",
    "-P", destination_folder,
    "--continue",
    "--quiet",
    "--no-check-certificate",
    url
  ])

def get_all_samples():
  os.makedirs('/content/data/fmri', exist_ok=True)
  !wget -P /content/data https://raw.githubusercontent.com/preprocessed-connectomes-project/abide/master/Phenotypic_V1_0b_preprocessed1.csv
  phenotypic_data = pd.read_csv('/content/data/Phenotypic_V1_0b_preprocessed1.csv')
  phenotypic_data['FILE_ID'].apply(download_sample)

In [ ]:
get_all_samples()

--2024-12-27 20:02:44--  https://raw.githubusercontent.com/preprocessed-connectomes-project/abide/master/Phenotypic_V1_0b_preprocessed1.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 449443 (439K) [text/plain]
Saving to: ‘/content/data/Phenotypic_V1_0b_preprocessed1.csv’

Phenotypic_V1_0b_pr 100%[===================>] 438.91K  --.-KB/s    in 0.05s   

2024-12-27 20:02:45 (9.44 MB/s) - ‘/content/data/Phenotypic_V1_0b_preprocessed1.csv’ saved [449443/449443]

